In [ ]:
# A jupter notebook that crawl Readme.md files from paperwithcode using python
# 1 Script to get the text from installation instructions of a readme file available in paperwithcode. For each file:
# 2. It extracts the content (code comments and text comments) from the section "Installation" from the readme. Please extract the part of text that relates or match with the following: |installation|setup|install|how|Getting started/quick start|
# 3. Create a dictionary with all the links and text per repository and output a csv file containing: url, text, tokenization of the text

In [16]:
import requests
import re

def getRepostitoryTopics(url, GitHub_Token, not_found):
    header = {'Authorization': 'Bearer ' + GitHub_Token}
    reposUrl = f"https://api.github.com/repos/{url}"
    reposr = requests.get(reposUrl, headers = header)
    reposj = reposr.json()
    try:
        topics = reposj["topics"]
        return (not_found, topics)
    except:
        return (not_found + 1, [])

In [15]:
repo_dict = {}
GitHub_Token = ""

In [36]:
listTopics = ['LLM']

In [38]:
def get_repositories_by_topic(topic):
    header = {'Authorization': 'Bearer ' + GitHub_Token}
    reposUrl = f"https://api.github.com/search/repositories?q=topic:{topic}&per_page=50"
    reposr = requests.get(reposUrl, headers = header)
    reposj = reposr.json()
    return reposj["items"]

# def get_repositories_by_topic(topic):
#     header = {'Authorization': 'Bearer ' + GitHub_Token}
#     reposUrl = f"https://api.github.com/search/repositories?q=topic:{topic}&per_page=50"
#     reposr = requests.get(reposUrl, headers=header)
    
#     # Check if the request was successful
#     if reposr.status_code == 200:
#         reposj = reposr.json()
#         # Check if 'items' key exists in the response
#         if "items" in reposj:
#             return reposj["items"]
#         else:
#             print(f"'items' key not found in response. Response JSON: {reposj}")
#             return []
#     else:
#         print(f"GitHub API request failed with status code: {reposr.status_code}")
#         return []

In [39]:
urls_with_topics = {}
for topic in listTopics:
    repositories_json = get_repositories_by_topic(topic)
    for repository in repositories_json:
        topics = repository["topics"]
        combined_topics_string = '\t'.join(topics)
        if(not "LLM" in combined_topics_string):
            urls_with_topics[repository["html_url"]] = topics

In [40]:
print(urls_with_topics)

{'https://github.com/ollama/ollama': ['go', 'golang', 'llama', 'llama2', 'llm', 'llms', 'mistral', 'ollama'], 'https://github.com/geekan/MetaGPT': ['agent', 'gpt', 'hacktoberfest', 'llm', 'metagpt', 'multi-agent'], 'https://github.com/run-llama/llama_index': ['agents', 'application', 'data', 'fine-tuning', 'framework', 'llamaindex', 'llm', 'rag', 'vector-database'], 'https://github.com/QuivrHQ/quivr': ['ai', 'api', 'chatbot', 'chatgpt', 'database', 'docker', 'frontend', 'html', 'javascript', 'llm', 'openai', 'postgresql', 'privacy', 'rag', 'react', 'rest-api', 'security', 'typescript', 'vector', 'ycombinator'], 'https://github.com/milvus-io/milvus': ['anns', 'cloud-native', 'distributed', 'embedding-database', 'embedding-similarity', 'embedding-store', 'faiss', 'golang', 'hnsw', 'image-search', 'llm', 'nearest-neighbor-search', 'tensor-database', 'vector-database', 'vector-search', 'vector-similarity', 'vector-store'], 'https://github.com/JushBJJ/Mr.-Ranedeer-AI-Tutor': ['ai', 'educati

In [45]:
import requests
import nltk
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def fetch_readme_content(repo_url):
    readme_url = repo_url.replace("github.com", "raw.githubusercontent.com") + "/main/README.md"
    response = requests.get(readme_url)
    return response.text if response.status_code == 200 else None
# todo: split text into sentences; tokenize the text; lemmatize and lowercase all tokens; remove stop words (preprocessing)

def extract_installation_instructions(readme_content):
    keywords = ["installation", "setup", "install", "how to", "getting started", "quick start"]
    pattern = re.compile("|".join(keywords), re.IGNORECASE)
    sections = re.split(r'#+ ', readme_content)
    installation_sections = [section for section in sections if pattern.search(section)]
    return installation_sections

# def tokenize_text(text):
#     return word_tokenize(text)
def process_text(text):
    sentences = sent_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    processed_sentences = []
    processed_tokens = []
    
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
        
        processed_sentences.append(' '.join(lemmatized_tokens))  # Join tokens to form the processed sentence
        processed_tokens.extend(lemmatized_tokens)  # Extend the list of processed tokens
    
    return processed_tokens, processed_sentences

def classify_complexity(text):
    complexity = -1  # Default complexity
    if any(word in text for word in ["pip install", "package manager install"]):
        complexity = 0
    elif any(word in text for word in ["container", "docker container", "docker compose up"]):
        complexity = 1
    elif any(word in text for word in ["from source", "git clone", ".git"]):
        complexity = 2
    return complexity

from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# def cluster_data(texts, topics, sentence_lengths, code_counts):
#     # Vectorize the texts and topics
#     vectorizer = TfidfVectorizer()
#     text_features = vectorizer.fit_transform(texts + topics).toarray()
    
#     # Combine all features
#     features = np.hstack((text_features, np.array(sentence_lengths).reshape(-1, 1), np.array(code_counts).reshape(-1, 1)))
    
#     # Optional: Apply PCA for dimensionality reduction
#     pca = PCA(n_components=0.95)  # Keep 95% of variance
#     reduced_features = pca.fit_transform(features)
    
#     # Cluster using Gaussian Mixture Model
#     gmm = GaussianMixture(n_components=3)  # We want to cluster into 3 groups
#     gmm.fit(reduced_features)
#     cluster_labels = gmm.predict(reduced_features)
#     probabilities = gmm.predict_proba(reduced_features)
    
#     return cluster_labels, probabilities

# Iterate through the URLs and perform the tasks
output_data = []

for url, topics in urls_with_topics.items():
    readme_url = url.replace("github.com", "raw.githubusercontent.com") + "/main/README.md"  # Define readme_url for each repository
    readme_content = fetch_readme_content(url)
    if readme_content:
        installation_instructions = extract_installation_instructions(readme_content)
        for instruction in installation_instructions:
            tokens = process_text(instruction)
            sentence = process_text(instruction)
            complexity = classify_complexity(instruction)
            # cluster_labels, probabilities = cluster_data([instruction], topics, [len(sentence)], [len(tokens)])
            output_data.append({
                "url": url,
                "readme_url": readme_url,
                "topic": topics,
                "text": instruction,
                'sentence': sentence,
                "token": tokens,
                # "cluster_labels": cluster_labels.tolist(),
                # "probabilities": probabilities.tolist()
                "level of complexity": complexity
            })

# Output to a JSON file
with open('data/corpus-topic3.json', 'w') as outfile:
    json.dump(output_data, outfile, indent=4)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
!pip install paperswithcode-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 4.6 MB/s eta 0:00:00
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.9/470.9 kB 5.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py)

In [82]:
## with paperswithcode

import json
import csv
import subprocess
from paperswithcode import PapersWithCodeClient
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

client = PapersWithCodeClient()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_readme_content(repo_url):
    readme_url = repo_url.replace("github.com", "raw.githubusercontent.com") + "/main/README.md"
    response = requests.get(readme_url)
    return response.text if response.status_code == 200 else None

def extract_installation_instruction(readme_content):
    # Call SOMEF via Docker to extract installation instructions
    command = f"docker run -i kcapd/somef describe -t 0.8 -o -"
    process = subprocess.Popen(command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate(input=readme_content.encode())
    if process.returncode == 0:
        return json.loads(stdout.decode())['installation instructions']
    else:
        print(stderr.decode())
        return None

def preprocess_data(text):
    sentences = sent_tokenize(text)
    tokens = [word_tokenize(sentence) for sentence in sentences]
    lemmatized_tokens = [[lemmatizer.lemmatize(token.lower()) for token in sentence if token.lower() not in stop_words] for sentence in tokens]
    return sentences, lemmatized_tokens

def classify_complexity(text):
    # Example complexity classification based on text length
    length = len(text)
    if length < 100:
        return 'Low'
    elif length < 500:
        return 'Medium'
    else:
        return 'High'

output_data = []

# Fetch repositories from PapersWithCode
repositories = client.repository_list()

for repo in repositories.results:
    repo_url = repo.url
    readme_content = get_readme_content(repo_url)
    if readme_content:
        installation_instructions = extract_installation_instruction(readme_content)
        if installation_instructions:
            sentences, tokens = preprocess_data(installation_instructions)
            complexity = classify_complexity(installation_instructions)
            output_data.append({
                "url": repo_url,
                "readme_url": repo_url.replace("github.com", "raw.githubusercontent.com") + "/main/README.md",
                "text": installation_instructions,
                "sentence": sentences,
                "tokens": tokens,
                "level_complexity": complexity
            })

# Output to JSON file
with open('output_data.json', 'w') as jsonfile:
    json.dump(output_data, jsonfile, indent=4)

# Output to CSV file
with open('output_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['url', 'readme_url', 'text', 'sentence', 'tokens', 'level_complexity']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for data in output_data:
        writer.writerow(data)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ccugutrillague/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:///Users/ccugutrillague/.docker/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.

docker: Cannot connect to the Docker daemon at unix:

In [52]:
output_data = []

# Fetch repositories from PapersWithCode
try:
    repositories = client.repositories.list()  # Assuming this is the correct method based on the pattern
except AttributeError:
    print("Failed to fetch repositories. Check if the 'paperswithcode-client' library has been updated.")
    # Handle the error or exit

for repo in repositories.results:
    repo_url = repo.repository_link
    # Continue with your logic here

Failed to fetch repositories. Check if the 'paperswithcode-client' library has been updated.


NameError: name 'repositories' is not defined

In [73]:

from paperswithcode import PapersWithCodeClient

client = PapersWithCodeClient()
papers = client.repository_list()
print(papers.next_page)

2


In [74]:
paper = papers.results[4]

In [75]:
paper

Repository(url='https://github.com/facebook/react', owner='facebook', name='react', description='The library for web and native user interfaces.', stars=218380, framework='none', is_official=None)